In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating new environment at `~/code/sanna21/lectures/certification/Project.toml`
    Updating registry at `~/.julia/registries/General`
  No Changes to `~/code/sanna21/lectures/certification/Project.toml`
  No Changes to `~/code/sanna21/lectures/certification/Manifest.toml`


In [26]:
using HomotopyContinuation
set_default_compile(:none)

:none

$\newcommand{\xn}[1]{x^{(#1)}}$
$\newcommand{\C}{\mathbb{C}}$
$\newcommand{\norm}[1]{\Vert #1 \Vert}$
$\newcommand{\IR}{\mathbb{IR}}$
$\newcommand{\IC}{\mathbb{IC}}$

# Lecture: Certification

So far, we discussed numerical methods to compute (all) isolated zeros of a polynomial system $F$.
Since we use numerical methods, we do not obtain exact zeros of $F$ but rather numerical approximations.

In [27]:
@var x y
F = System([x^2 + y^2 - 1; 2x - 3y + 1])
S = solutions(solve(F))

2-element Vector{Vector{ComplexF64}}:
 [-0.9532542188779434 + 1.504632769052528e-36im, -0.30216947925196225 + 7.52316384526264e-37im]
 [0.6455619111856358 + 1.232595164407831e-32im, 0.7637079407904238 + 6.162975822039155e-33im]

In [28]:
F(S[2])

2-element Vector{ComplexF64}:
 1.1102230246251565e-16 + 2.532775694944723e-32im
                    0.0 + 6.162975822039155e-33im

## Certification and bounds

For many applications, this is sufficient.


But for some applications, in particular in pure mathematics, we want to **certify** that the obtained approximations correspond to actual zeros of $F$.

Additionally, we want to certify that we found a certain number of distinct zeros, establishing a **lower bound** on the number of solutions of $F$.

**Example:**
* There exists an instance of five plance conics such that there are 3264 conics that are tangent to all five given conics. Now I claim to have found such an instance of five conics. How can I proof my claim? See [3264 Conics in a second](https://www.ams.org/journals/notices/202001/rnoti-p30.pdf) for a possible answer.

* In bio-chemical reaction networks it is of interest to show the existance of parameter values where there are (multiple) positive steady states.

It is of great interest to also certify that we found *all* isolated solutions of $F$.
Unfortunately, this is so far only possible if an upper bound is already established and the lower bound obtained from the certification routine matches the upper bound. Establishing an upper bound for a polynomial system based on a computational method that does not involve Gröbner basis computations is an **important open problem**.


### Trace test
A numerical method to test whether all solutions are found is given by the *trace test* (as discusssed in Taylor's lecture).
However, the trace test does not produce a rigorous certificate.

The result of the trace test can be interpreted similarly to the numerical computation of the smallest eigenvalue of a matrix. If the computed smallest eigenvalue is on the order of the machine precision, then you probably conclude that the matrix is singular. But this is *not* a proof (or certificate) that the matrix is singular.



## Certification in action

**Example**:

Our system `F` above has total degree 2 and therefore at most 2 isolated solutions.
Our numerical computation indicates that it has in fact 2 real regular isolated solutions. But can we **proof** this?

In [29]:
cert = certify(F, S)

CertificationResult
• 2 solution candidates given
• 2 certified solution intervals (2 real, 0 complex)
• 2 distinct certified solution intervals (2 real, 0 complex)

The `certify` routine performs a computation which delivers a rigouros proof that the system `F` has exactly 2 regular isolated solutions. It also provides us interval boxes that contain the true solutions.

In [17]:
certificates(cert)[1]

SolutionCertificate:
solution_candidate = [
  0.6455619111856358 + 0.0im,
  0.7637079407904237 + 0.0im,
]
is_certified = true
certified_solution_interval = [
  [0.645561911186 +/- 6.39e-13] + [+/- 2.74e-13]im,
  [0.763707940790 +/- 6.38e-13] + [+/- 2.14e-13]im,
]
precision = 53
is_real = true
index = 1

In [19]:
certificates(cert)[2]

SolutionCertificate:
solution_candidate = [
  -0.9532542188779434 + 0.0im,
  -0.30216947925196225 + 0.0im,
]
is_certified = true
certified_solution_interval = [
  [-0.953254218878 +/- 1.71e-13] + [+/- 1.14e-13]im,
  [-0.302169479252 +/- 2.13e-13] + [+/- 1.75e-13]im,
]
precision = 53
is_real = true
index = 2

In the following we will explain what `certify` does in more detail. 

## Certification Methods

We focus on two strategies to certify solutions to *square* polynomial systems:

* **Smale's $\alpha$-theory** and
* **Krawczyk's method**.

Both methods can certify that Newton's method starting from a given point converges super-linearly to a zero of the system.

The restriction to square polynomial systems is necessary since to certify solutions to overdetermined systems additional global information is necessary.
In [[DHS20]](https://arxiv.org/abs/1812.02851), the authors develop various techniques to certify overdetermined systems requiring different global information.

Certification based on Smale's $\alpha$-theory is implemented in the software [alphaCertified](https://arxiv.org/abs/1011.1091).

The `certify` method we saw earlier in action is based on Krawczyk's method and implemented in `HomotopyContinuation.jl`. See [arxiv:2011.05000](https://arxiv.org/abs/2011.05000) for an article about the implementation.


### Smale's $\alpha$-theory

Smale's $\alpha$-theory only requires data from one point, which is very valuable for the theory of computation.
In particular, it is the building block for the complexity analysis of polynomial homotopy continuation methods [Beltran:Pardo:2009, Buergisser:Cucker:2011, Beltran:Leykin:2013, Lairez:2017] and certified path tracking algorithms [Beltran:Leykin:2013].

In [Smale:1986], Smale introduced the notion of an *approximate zero*, the $\alpha$-number and the $\alpha$-theorem.
For a square polynomial system $F$ in $n$ variables, consider the Newton iteration

\begin{equation*}
\begin{array}{rl}
J_F(\xn{j}) \Delta \xn{j} &= F(\xn{j}) \\
\xn{j+1} &= \xn{j} - \Delta \xn{j} \\ 
\end{array}
, \quad j=0,1,2,\ldots
\end{equation*}

starting at the initial guess $\xn{0} \in \C^n$ where $J_F$ is the Jacobian of $F$.
An approximate zero of $F$ is any point $\xn{0} \in \mathbb{C}^n$ such that Newton's method converges quadratically towards a zero of $F$.
This means that the number of correct significant digits roughly doubles with each iteration of Newton's method.

#### Definition: Approximate zero
The point $\xn{0} \in \C^n$ is an approximate zero of $F$ if the Newton iterates $\xn{j}$ are defined for $j = 1,2,\ldots$ and satisfy

$$
\norm{\Delta \xn{j}} \le \left( \frac12 \right)^{2^j - 1} \norm{\Delta \xn{0}}\,. 
$$

If $\xn{0}$ is an approximate zero, then the true zero $x^* \in \C^n$ of $F$ to that the iterates are converging is the *associated zero* of $\xn{0}$.

![convergence](convergence.png)

#### Smale's $\alpha$-theorem

Smale's \(\alpha\)-theorem gives a sufficient condition for $\xn{0}$ to be an approximate zero.
The theorem uses

\begin{align}\label{eq:smale_gamma}
\gamma(F,x) &\,\,=\,\, \sup_{k\geq 2}\big\Vert \frac{1}{k!}\,J_F(x)^ {-1} D^kF(x)\big\Vert^\frac{1}{k-1}\ \text{ and } \beta(F,x) \,\,=\,\, \Vert J_F(x)^{-1}F(x)\Vert
\end{align}

where $D^kF$ is the tensor of order-$k$ derivatives of \(F\) and the tensor $J_F^{-1}D^kF$ is understood as a 
multilinear map $A:(\mathbb{C}^n)^k\to \mathbb{C}^n$.

**Theorem**
There is a naturally defined number $\alpha_0$ approximately equal to
0.1307 such that if $$\alpha(F, \xn{0}) := \beta(F,\xn{0}) \, \gamma(F,\xn{0}) <~\alpha_0,$$ then $\xn{0}$ is an approximate zero of $F$.

To avoid the computation of the $\gamma$-number,
Shub and Smale [Shub:Smale:1993] derived an upper bound for $\gamma(F,x)$ that can be computed exactly and efficiently.
Hence, one can decide algorithmically whether $x$ is an approximate zero using only the data of the point~$x$ itself and \(F\). 
Hauenstein and Sottile implemented these ideas in the software `alphaCertified`.

## Krawczyk's method

Before we can talk about Krawczyks' method in detail we need to talk about about interval arithmetic.

### Interval arithmetic

Since the 1950s researchers (see ,e.g., the work of Moore) have worked on interval arithmetic.
Interval arithmetic allows certified computations while still using floating-point arithmetic.


**Real interval arithmetic**

Real interval arithmetic concerns computing with compact real intervals. Following \cite{Mayer:2017} we denote the set of all compact real intervals by

$$\mathbb I\mathbb R:= \{[a,b]\mid a,b\in\mathbb R, a\leq b\}.$$

For $X, Y \in \IR$ and the binary operation $\circ \in \{ +,-, \cdot, / \}$, we define

\begin{equation}\label{cert:real_interval_algebra}
X \circ Y= \{ x \circ y \,|\, x\in X,y\in Y\}
\end{equation}

where we assume $0 \notin Y$ in the case of division.

$\IR$ has an interesting algebraic structure:

* $(\IR, +)$ is a commutative semigroup with neutral element.
* $(\IR, +, \cdot)$ has no zero divisors.
Furthermore, if $I,J,K,L\in\IR$, then
* $I\cdot (J + K) \subseteq I\cdot J + I\cdot K$, but equality does not hold in general.
* $I\subseteq J, K\subseteq L$, then $I \circ K\subseteq J\circ L$ for $\circ\in\{+,-,\cdot, /\}$.

Working with interval arithmetic is challenging because of the third item from the previous theorem: **distributivity does not hold in $\IR$**

In [15]:
using HomotopyContinuation.IntervalArithmetic: Interval, IComplex

In [6]:
x = Interval(-1, 0)
y = Interval(1, 1)
z = Interval(0, 1)
@show x y z;

x = -0.5 ± 0.5
y = 1.0 ± 5e-324
z = 0.5 ± 0.5


In [7]:
x + y

0.5000000000000001 ± 0.5

In [5]:
(x + y) * z

0.5000000000000002 ± 0.5

Here is an example to illustrate the lack of distributivity:

In [40]:
x * z + y * z

0.0 ± 1

**Complex Interval Arithmetic**

We define the set of *rectangular complex intervals* as

$$\IC :=\{X + i Y\mid X,Y\in\IR\}$$

where $X+iY = \{x+iy\mid x\in X,y\in Y\}$ and $i = \sqrt{-1}$.
We define the algebraic operations for $I = X+iY, J=W+iZ\in\IC$ in terms of operations on the real intervals:

\begin{alignat}{2}\label{cert:complex_arithmetic}
I + J &:= (X + W) + i (Y+Z),\qquad I \cdot J &&:= (X\cdot W - Y \cdot Z) + i (X\cdot Z + Y\cdot W)\\\nonumber
I - J &:= (X - W) + i (Y-Z),\qquad \;\, \frac{I}{J} &&:= \frac{X \cdot W + Y\cdot Z}{W\cdot W + Z\cdot Z} + i \frac{Y \cdot W - X\cdot Z}{W\cdot W + Z\cdot Z}
\end{alignat}

In [21]:
u = IComplex(Interval(-1, 0), Interval(-0.5, 0.5))
v = IComplex(Interval(-1, 1), Interval(-0.25, 0.25))
@show u v u * v u + v;

u = (-0.5 ± 0.5) + (0.0 ± 0.5)im
v = (0.0 ± 1) + (0.0 ± 0.25)im
u * v = (0.0 ± 1.125) + (0.0 ± 0.75)im
u + v = (-0.5000000000000001 ± 1.5) + (0.0 ± 0.75)im


$\IC$ has the same algebraic structure as $\IR$. In particular, no distributivity!

This lack of distributivity has an import consequence for the evaluation of polynomials in $\IC$!

The evaluation of polynomial maps $F: \IC^n \to \IC$ is only well-defined if $F$ is defined by a straight-line program, and not just by a list of coefficients.

This is one of the reasons, we define systems using symbolic expressions.

In [41]:
@var a b c

(a, b, c)

In [43]:
(a + b) * c

c*(a + b)

In [56]:
expand((a + b) * c)

a*c + b*c

Internally, every symbolic expression is turned into a list of instructions

In [71]:
instr, _ = ModelKit.instruction_list([a * c + b * c])
instr

ι_1 = INSTR_MUL(b, c)
ι_2 = INSTR_MULADD(a, c, ι_1)


In [24]:
F

System of length 2
 2 variables: x, y

 -1 + x^2 + y^2
 1 + 2*x - 3*y

In [31]:
show_straight_line_program(cert)

c = [-1,-3,1,2]
t[1] = x[1]
t[2] = x[2]
t[3] = SQR(t[2])
t[4] = ADD(t[3],c[1])
t[3] = SQR(t[1])
t[5] = ADD(t[3],t[4])
t[3] = MULADD(c[2],t[2],c[3])
t[4] = MULADD(c[4],t[1],t[3])
out[1] = t[5]
out[2] = t[4]


### Krawcyzk's method (now really)

In 1969, Krawczyk developed an **interval arithmetic version of Newton's method**. Later in 1977, Moore recognized that Krawczyk's method can be used to **certify the existence and uniqueness** of a solution to a system of nonlinear equations.

<div style="border:2px solid gray; padding: 0.5em; back:center"> 
    <strong>Note:</strong>
    The results in this section are stated for square polynomial systems but they hold equally for square systems of rational functions.
    Krawczyk's method is even valid for general square systems of analytic functions.
</div>



First, we need three definitions.

#### Interval enclosure
Let $F: \mathbb{C}^n \rightarrow \mathbb{C}^{n}$ be a system of polynomials. A map $\square F: \mathbb{IC}^n \rightarrow \mathbb{IC}^{n}$ is an interval enclosure of the system $F$ if for every $I \in \mathbb{IC}^n$ we have $\{F(x) \mid x \in I\} \subseteq \square F(I).$


We use the notation $\square F$ to denote the interval enclosure of $F$.

#### Interval matrix norm

Let $A\in\mathbb{IC}^{n\times n}$. We define the operator norm of $A$ as
$\Vert A\Vert_\infty := \max\limits_{B\in A} \max\limits_{v\in\mathbb C^n} \tfrac{\Vert Bv\Vert_\infty}{\Vert v\Vert_\infty},$
where $\Vert (v_1,\ldots,v_n) \Vert_\infty = \max_{1\leq i\leq n}\vert v_i\vert$ is the infinity norm in $\mathbb C^n$.


#### Krawczyk operator

Let $F: \mathbb{C}^n \rightarrow \mathbb{C}^n$ be a system of polynomials, and $\mathrm{J}F$ be its Jacobian matrix seen as a function $\mathbb C^n \to \mathbb C^{n\times n}$.
Let $\square F$ be an interval enclosure of $F$ and $\square \mathrm{J}F$ be an interval enclosure of $\mathrm{J}F$.
Furthermore, let $I \in \mathbb{IC}^n$ and $x \in \mathbb C^n$ and let $Y \in \mathbb C^{n\times n}$ be an invertible matrix.
We define the Krawczyk operator

\begin{align*}
K_{x,Y}(I) := x - Y \cdot \square F(x) + (\mathbf 1_n - Y \cdot \square \mathrm{J}F(I))(I-x).
\end{align*}

Here, $\mathbf 1_n$ is the ${n\times n}$-identity matrix.




#### Theorem 1
Let $F: \mathbb{C}^n \rightarrow \mathbb{C}^n$ be a system of polynomials and $I\in\IC^n$.
Let $x\in I$ and $Y\in\mathbb{C}^{n\times n}$ be an invertible complex $n\times n$ matrix.
The following holds.
* If $K_{x,Y}(I) \subset I$, there is a zero of $F$ in $I$.
* If additionally $\sqrt{2} \, \lVert \mathbf 1_n - Y \square \mathrm{J}F(I) \rVert_\infty < 1$, then $F$ has exactly one zero in $I$.

To simplify our language when talking about intervals $I \in \IC^n$ satisfying Theorem 1, we introduce the following definitions.

In [54]:
C₁, C₂ = certificates(cert);

In [35]:
certified_solution_interval(C₁)

2×1 Arblib.AcbMatrix:
 [-0.953254218878 +/- 1.71e-13] + [+/- 1.14e-13]im
 [-0.302169479252 +/- 1.52e-13] + [+/- 1.14e-13]im

In [36]:
certified_solution_interval_after_krawczyk(C₁)

2×1 Arblib.AcbMatrix:
 [-0.953254218877943 +/- 8.16e-16] + [+/- 4.49e-26]im
 [-0.302169479251962 +/- 5.81e-16] + [+/- 3.00e-26]im

### Some useful language

#### Definition: Interval approximate zero
 Let $F: \C^n \rightarrow \C^n$ be a square system of polynomials and $I \in \IC^n$. Let $K_{x,Y}(I)$ be the associated Krawczyk operator (see Definition \ref{cert:def_krawczyk}).
 If there exists an invertible matrix $Y\in\mathbb C^{n\times n}$ such that $K_{x,Y}(I)\subset I$, we say that $I$ is an *interval approximate zero* $F$.
 We call $I$ a *strong interval approximate zero* of $F$ if in addition $\sqrt{2} \lVert \mathbf 1_n - Y \square \mathrm{J}F(I) \rVert_\infty < 1$.

#### Definition: Interval associated zero
 If $I$ is an interval approximate zero, then, by Theorem 1, $I$ contains a zero of $F$. We call such a zero an \emph{associated zero} of $I$.
 If $I$ is a strong interval approximate zero, then there is a unique associated zero and we refer to is as *the* associated zero of $I$.


The notion of strong interval approximate zero is stronger than the definition suggests at first sight. We do not only certify that a unique zero of $F$ exists inside $I$ but we even certify that we can approximate this zero with arbitrary precision.

### Reality of zeros

We can also certify the reality of a zero.

#### Real zero lemma
 Let $F: \C^n \rightarrow \C^n$ be a real square system of polynomials and $I \in \IC^n$ a strong interval approximate zero of $F$. 
Then there exists $x\in I$ and $Y\in \C^{n\times n}$ satisfying $K_{x,Y}(I) \subset I$ and $\sqrt{2} \, \lVert \mathbf 1_n - Y \square \mathrm{J}F(I) \rVert_\infty < 1$.
 If additionally $\{ \bar{z} \, | \, z \in K_{x,Y}(I) \} \subset I$, then the associated zero of $I$ is real.

In [50]:
certified_solution_interval(C₁)

2×1 Arblib.AcbMatrix:
 [-0.953254218878 +/- 1.71e-13] + [+/- 1.14e-13]im
 [-0.302169479252 +/- 1.52e-13] + [+/- 1.14e-13]im

In [55]:
certified_solution_interval(C₂)

2×1 Arblib.AcbMatrix:
 [0.645561911186 +/- 5.64e-13] + [+/- 1.99e-13]im
 [0.763707940790 +/- 5.75e-13] + [+/- 1.51e-13]im

In [56]:
is_real(C₁)

true

For a wide range of applications, positive real zeros are of particular interest.
#### Positive real zero lemma
 Let $F: \C^n \rightarrow \C^n$ be a real square system of polynomials and $I \in \IC^n$ a strong interval approximate zero of $F$ satisfying the conditions of the real zero lemma.
 If $\mathrm{Re}(I) > 0$, then the associated zero of $I$ is real and positive.

In [46]:
is_positive(C₁)

false

In [57]:
is_positive(C₂)

true

#### Distinct zeros

Given two strong interval arithmetic zeros $I$ and $J$ we know that the associated zeros are distinct if $I \cap J = \emptyset$.
Using this oberservation we can produce a list of strong interval arithmetic zeros with distinct associated zeros. This establishes a **lower bound** on the number of (real) solutions of the system.

In [77]:
cert

CertificationResult
• 2 solution candidates given
• 2 certified solution intervals (2 real, 0 complex)
• 2 distinct certified solution intervals (2 real, 0 complex)